In [ ]:
import torch 
import torch.nn as nn
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
class SimpleSignal(torch.utils.data.Dataset):
    def __init__(self, N=1000, T=100):
        super(SimpleSignal, self).__init__()
        self.N = N
        self.T = T
        self.data, self.labels = self.generate_data()
        self.train_ix, self.test_ix = self.get_ix_splits()
    
    def __getitem__(self, ix):
        return self.data[ix], self.labels[ix]
    
    def __len__(self):
        return len(self.data)

    def get_ix_splits(self):
        split_props = [0.8, 0.1, 0.1] # Train/validation/test split proportions
        indices = range(len(self.data))
        split_points = [int(len(self.data)*i) for i in split_props]
        train_ix = np.random.choice(indices,
                                    split_points[0],
                                    replace=False)
        test_ix = np.random.choice((list(set(indices)-set(train_ix))),
                                    split_points[1],
                                    replace=False)
        return train_ix, test_ix

    def generate_data(self):
        N_pos = int(self.N/2)
        N_neg = int(self.N/2)
        data = [] 
        labels = [] 
        for i in range(N_pos + N_neg):
            time_series = np.zeros(self.T)
#             time_series = np.random.normal(0, .3, self.T) # You can make the task harder if you raise the standard deviation of the noise
            if i < N_pos:
                labels.append(0)
            else:
                time_series[np.random.choice(self.T)] = 1 # Put the signal at a random place
                labels.append(1)
            data.append(time_series)
            
        # --- shuffle the data ---
        permutation = np.random.choice(len(data), len(data), replace=False).astype(int)
        data = [data[i] for i in permutation]
        labels = [labels[i] for i in permutation]
        return torch.tensor(data, dtype=torch.float).unsqueeze(2), torch.tensor(labels, dtype=torch.long)

In [ ]:
HIDDEN_DIMENSION = 20
N_LAYERS = 1
BATCH_SIZE = 10
N_EPOCHS = 20
LEARNING_RATE = 0.01
SEQ_LENGTH = 10
N_INSTANCES = 1000
N_FEATURES = 1
N_CLASSES = 2

# --- create the dataset ---
data = SimpleSignal(T=SEQ_LENGTH, N=N_INSTANCES)

# --- define the data loaders ---
train_sampler = SubsetRandomSampler(data.train_ix) # Random sampler for training indices
test_sampler = SubsetRandomSampler(data.test_ix) # Random sampler for testing indices

train_loader = torch.utils.data.DataLoader(dataset=data,
                                           batch_size=BATCH_SIZE, 
                                           sampler=train_sampler,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=data,
                                          batch_size=BATCH_SIZE, 
                                          sampler=test_sampler,
                                          shuffle=False)

# --- define your model here ---
class RNN(nn.Module):
    def __init__(self, HIDDEN_DIMENSION, N_CLASSES, N_FEATURES):
        super(RNN, self).__init__()
        # --- define mappings here ---
    
    def forward(self, X):
        # --- define forward pass here ---
        return prediction
        

# --- initialize the model and the optimizer ---
model = RNN(HIDDEN_DIMENSION, N_CLASSES, N_FEATURES)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) # Using the Adam optimizer - don't worry about the details, it's going to update the network's weights.

# --- training the model ---
for epoch in range(N_EPOCHS):
    for i, (time_series, labels) in enumerate(train_loader): # Iterate through the training batches
        # --- Forward pass ---
        predictions = model(time_series)
        
        # --- Compute gradients and update weights ---
        optimizer.zero_grad()
        loss = model.applyLoss(predictions, labels) # I make applyLoss an internal function in my networks, you can do it however you want
        loss.backward()
        optimizer.step()
        if (i+1) % 1 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, N_EPOCHS, i+1, len(train_loader), loss.item()))

# Voila! Your model is trained! Now run the same model on the testing dataset and report the accuracy of the model!